# Sprint 機械学習スクラッチ 線形回帰

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
np.random.seed(0)

X_array = np.array([[1, 1], [1, 2], [1, 3]])
y_array = np.ones(3)

In [24]:
class ScratchLinearRegression():
    """
    線形回帰のスクラッチ実装

    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    no_bias : bool
      バイアス項を入れない場合はTrue
    verbose : bool
      学習過程を出力する場合はTrue

    Attributes
    ----------
    self.coef_ : 次の形のndarray, shape (n_features,)
      パラメータ
    self.loss : 次の形のndarray, shape (self.iter,)
      訓練データに対する損失の記録
    self.val_loss : 次の形のndarray, shape (self.iter,)
      検証データに対する損失の記録

    """

    def __init__(self, num_iter, lr, no_bias, verbose):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.no_bias = no_bias
        self.verbose = verbose
        # 損失を記録する配列を用意
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        #np.random.seed(0)

    def fit(self, X, y, X_val=None, y_val=None):
        """
        線形回帰を学習する。検証データが入力された場合はそれに対する損失と精度もイテレーションごとに計算する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        np.random.seed(0)
        self.theta = np.random.randn(X.shape[1])

        self.X = np.copy(X)
        self.y = np.copy(y)
        self.X_val = np.copy(X_val)
        self.y_val = np.copy(y_val)
        
        if self.verbose:
            # verboseをTrueにした際は学習過程を出力
            print()

        for i in range(self.iter):
            self.linear_hypo = self._linear_hypothesis(self.X)
            #print(self.linear_hypo)
            
            self.grad = self._gradient_descent(self.X, self.y)
            #print("[theta]", self.grad)
            
            self.pred = self.predict(self.X)
            #print("[pred]", self.pred)
            
            self.loss[i] = self.loss_func(self.y)
            print("loss", self.loss)
            
            #if X_val != None and y_val != None:
                #self.val_loss[i] = self.loss(self.y_val)

            #grad = self._gradient_descent(X, y, alpha=0.1, error=0)  
            
            #print(grad)

    def _linear_hypothesis(self, X):
        """
        線形の仮定関数を計算する

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
          訓練データ

        Returns
        -------
          次の形のndarray, shape (n_samples, 1)
          線形の仮定関数による推定結果

        """
        self.hypo = np.dot(self.X, self.theta.T)

        return self.hypo

    def _gradient_descent(self, X, y, alpha=0.1, error=0):
        """
        最急降下法

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
          訓練データ

        alpha：学習率

        Returns
        -------

        """
        # m = len(self.y)
        self.theta = self.theta - alpha*(1/2)*np.dot(self.X.T,(self._linear_hypothesis(self.X)-y))
        
        # theta = np.random.random(2)
        
        #for x in range(X.shape[0]):
        # self.temp0 = theta[0] - alpha*(1/(m))*np.sum(theta[0]+theta[1]*X-y)
        # self.temp1 = theta[1] - alpha*(1/(m))*np.sum((theta[0]+theta[1]*X-y)*X)
        
        # self.gradient += alpha * (self._linear_hypothesis(X)[x] - y[x]) * X[x, :]

        # self.gradient = self.gradient / X.shape[0]

        # thetaの更新
        # theta = np.array([self.temp0, self.temp1])
        
        return self.theta

    def predict(self, X):#self.linear_hypo
        """
        線形回帰を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            線形回帰による推定結果
        """
        # print(self.X)

        pred = np.dot(self.X, self._gradient_descent(self.X, self.y).T)

        return pred
    
    def loss_func(self, y):
        """
        平均二乗誤差の計算

        Parameters
        ----------
        y_pred : 次の形のndarray, shape (n_samples,)
          推定した値
        y : 次の形のndarray, shape (n_samples,)
          正解値

        Returns
        ----------
        mse : numpy.float
          平均二乗誤差
        """
        loss = np.average((self.pred - y) ** 2) / 2

        return loss

In [25]:
scr_lr = ScratchLinearRegression(num_iter=15, 
                                 lr=1, 
                                 no_bias=1, 
                                 verbose=False, )

In [26]:
scr_lr.fit(X_array, y_array)

loss [0.01719295 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
loss [0.01719295 0.01505335 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
loss [0.01719295 0.01505335 0.013996   0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
loss [0.01719295 0.01505335 0.013996   0.01301361 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
loss [0.01719295 0.01505335 0.013996   0.01301361 0.01210017 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
loss [0.01719295 0.01505335 0.013996   0.01301361 0.01210017 0.01125085
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
loss [0.01719295 0.01505335 0.013996   0.013

## 【問題1】仮定関数
以下の数式で表される線形回帰の仮定関数を実装してください。メソッドの雛形を用意してあります。


h
θ
(
x
)
=
θ
0
x
0
+
θ
1
x
1
+
.
.
.
+
θ
j
x
j
+
.
.
.
+
θ
n
x
n
.
(
x
0
=
1
)

$x$ : 特徴量ベクトル


$\theta$ : パラメータベクトル


$n$ : 特徴量の数


$x_j$ : j番目の特徴量


$\theta_j$ : j番目のパラメータ（重み）


特徴量の数$n$は任意の値に対応できる実装にしてください。


なお、ベクトル形式で表すと以下のようになります。


h
θ
(
x
)
=
θ
T
⋅
x
.

雛形


クラスの外から呼び出すことがないメソッドのため、Pythonの慣例としてアンダースコアを先頭にひとつつけています。



In [72]:
def _linear_hypothesis(X):
    """
    線形の仮定関数を計算する

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ

    Returns
    -------
      次の形のndarray, shape (n_samples, 1)
      線形の仮定関数による推定結果

    """
    line_hypo = np.dot(X, theta.T)
    
    return line_hypo

In [73]:
li_hy = _linear_hypothesis(X_array)

## 【問題2】最急降下法
最急降下法により学習させる実装を行なってください。以下の式で表されるパラメータの更新式のメソッド_gradient_descentを追加し、fit
メソッドから呼び出すようにしてください。


θ
j
:=
θ
j
−
α
1
m
m
∑
i
=
1
 
[
(
h
θ
(
x
(
i
)
)
−
y
(
i
)
)
x
(
i
)
j
]

$\alpha$ : 学習率


$i$ : サンプルのインデックス


$j$ : 特徴量のインデックス


雛形


ScratchLinearRegressionクラスへ以下のメソッドを追加してください。コメントアウト部分の説明も記述してください。



In [65]:
def _gradient_descent_1(X, y, alpha=0.1, error=0):
    """
    最急降下法

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    
    alpha：学習率

    Returns
    -------
    
    """
    # データフレーム型だった場合にnd_array型に変換
    if type(X) is pd.core.frame.DataFrame:
        X = X.values
        
    np.random.seed(0)
    theta = np.random.random(size=X.shape[1])

    y = np.ones(5)
    
    # gradientを求める
    gradient = 0
    for x in range(X.shape[1]):
        gradient += alpha * (_linear_hypothesis(X)[x] - y[x]) * X[x, :]
    
    gradient = gradient / X.shape[1]
    
    # thetaの更新
    error = theta - gradient
    
    return error

In [38]:
result = _gradient_descent(X_array, y_array, alpha=0.1, error=0)

result

array([0.48665375, 0.60407   ])

## 【問題3】推定
推定する仕組みを実装してください。ScratchLinearRegressionクラスの雛形に含まれるpredictメソッドに書き加えてください。


仮定関数 $h_\theta(x)$ の出力が推定結果です。



In [248]:
def predict(X):
    """
    線形回帰を使い推定する。

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
        サンプル

    Returns
    -------
        次の形のndarray, shape (n_samples, 1)
        線形回帰による推定結果
    """

    theta = np.random.random(size=X.shape[1])
    theta = np.dot(X, _gradient_descent(X).T)    
    
    return theta

In [250]:
pred = predict(X_array)
pred

array([-0.53251795, -1.35858852, -3.01072967, -4.66287081, -6.31501196])

## 【問題4】平均二乗誤差
線形回帰の指標値として用いられる平均二乗誤差（mean square error, MSE）の関数を作成してください。


平均二乗誤差関数は回帰問題全般で使える関数のため、ScratchLinearRegressionクラスのメソッドではなく、別の関数として作成してください。雛形を用意してあります。


平均二乗誤差は以下の数式で表されます。


L
(
θ
)
=
1
m
m
∑
i
=
1
 
(
h
θ
(
x
(
i
)
)
−
y
(
i
)
)
2
.

$m$ : 入力されるデータの数


$h_\theta()$ : 仮定関数


$x^{(i)}$ : i番目のサンプルの特徴量ベクトル


$y^{(i)}$ : i番目のサンプルの正解値


なお、最急降下法のための目的関数（損失関数）としては、これを2で割ったものを使用します。（問題5, 9）

gradient = 0
y = np.ones(5)
alpha = 0.1

for i in range(1, 3):
    cnt = 1
    for x in range(X_array.shape[0]):
        gradient += X_array[x] - alpha *(_linear_hypothesis(X_array, theta_array)[i] - y[i]) * X_array[i]
    cnt +=1

gradient = gradient / cnt

print(gradient)

y_array = np.ones(5)

alpha * (_linear_hypothesis(X=X_array, theta=theta_array)[1] - y[1])

_linear_hypothesis(X=X_array, theta=theta_array)[1] - y[1]

gradient_1 = X_array[1] - alpha *(_linear_hypothesis(X_array, theta_array)[1] - y[1]) * X_array[1] 
gradient_2 = X_array[2] - alpha *(_linear_hypothesis(X_array, theta_array)[2] - y[2]) * X_array[2]

_linear_hypothesis(X=X_array, theta=theta_array)[2] - y[2]

X_array[2]

theta_array.shape[1]

theta_array.shape[2]

## 【問題4】平均二乗誤差
線形回帰の指標値として用いられる平均二乗誤差（mean square error, MSE）の関数を作成してください。


平均二乗誤差関数は回帰問題全般で使える関数のため、ScratchLinearRegressionクラスのメソッドではなく、別の関数として作成してください。雛形を用意してあります。


平均二乗誤差は以下の数式で表されます。



In [47]:
def MSE(y_pred, y):
    """
    平均二乗誤差の計算

    Parameters
    ----------
    y_pred : 次の形のndarray, shape (n_samples,)
      推定した値
    y : 次の形のndarray, shape (n_samples,)
      正解値

    Returns
    ----------
    mse : numpy.float
      平均二乗誤差
    """
    mse = np.average((y_pred - y) ** 2)
    
    return mse

## 【問題5】目的関数
以下の数式で表される線形回帰の 目的関数（損失関数） を実装してください。そして、これをself.loss, self.val_lossに記録するようにしてください。


目的関数（損失関数） 
J
(
θ
)
 は次の式です。


J
(
θ
)
=
1
2
m
m
∑
i
=
1
 
(
h
θ
(
x
(
i
)
)
−
y
(
i
)
)
2
.

m
 : 入力されるデータの数


h
θ
(
)
 : 仮定関数


x
(
i
)
 : i番目のサンプルの特徴量ベクトル


y
(
i
)
 : i番目のサンプルの正解値

In [56]:
def loss(y_pred, y):
    """
    平均二乗誤差の計算

    Parameters
    ----------
    y_pred : 次の形のndarray, shape (n_samples,)
      推定した値
    y : 次の形のndarray, shape (n_samples,)
      正解値

    Returns
    ----------
    mse : numpy.float
      平均二乗誤差
    """
    
    loss = np.average((y_pred - y) ** 2) / 2
    
    return loss

In [58]:
def val_loss(y_true, y):
    """
    平均二乗誤差の計算

    Parameters
    ----------
    y_pred : 次の形のndarray, shape (n_samples,)
      推定した値
    y : 次の形のndarray, shape (n_samples,)
      正解値

    Returns
    ----------
    mse : numpy.float
      平均二乗誤差
    """
    
    val_loss = np.average((y_true - y) ** 2) / 2
    
    return val_loss    
    
    

In [54]:
np.random.seed(100)
y_1 = np.random.random(5)
y_1

array([0.54340494, 0.27836939, 0.42451759, 0.84477613, 0.00471886])

In [45]:
y_2 = np.ones(5)

In [59]:
y_3 = np.array([0,1,0,1,1])

In [46]:
err = np.average((y_2 - y_1) ** 2)
err

0.41501775984547484

In [55]:
MSE(y_1, y_2)

0.41501775984547484

In [57]:
loss(y_1, y_2)

0.20750887992273742

In [62]:
val_loss(y_3, y_2)

0.2

## 【問題6】学習と推定
機械学習スクラッチ入門のSprintで用意したHouse Pricesコンペティションのデータに対してスクラッチ実装の学習と推定を行なってください。


scikit-learnによる実装と比べ、正しく動いているかを確認してください。

## 【問題7】学習曲線のプロット
学習曲線を表示する関数を作成し、実行してください。グラフを見て損失が適切に下がっているかどうか確認してください。


線形回帰クラスの雛形ではself.loss, self.val_lossに損失を記録しておくようになっているため、入力にはこれを利用してください。